In [1]:
import os 

os.chdir('/home/daiv8314/vsd-linux')

In [2]:
import json
logs=[]

bash_commands = []

with open('log_data/31-December.json', 'r') as f:
    for line in f:
        logs.append(json.loads(line)['content'])
        
        
with open('bash_commands.json', 'r') as f:
    for line in f:
        bash_commands.append(json.loads(line)['command'])
        


In [3]:
from project.utils.handy_functions import add_full_command_to_log

for log in logs:
    log['timestamp'] = float(log['timestamp'])
    if log['command'] in bash_commands:
        log['is_bash'] = 1
    else:
        log['is_bash'] = 0
    
    log = add_full_command_to_log(log)
    

In [9]:
from pymongo import MongoClient
db = MongoClient("mongodb://localhost:27017/")['kursinis']
col = db['logs']
embedded_col = db['embedded_logs']


In [6]:
col.insert_many(logs[:4000])

InsertManyResult([ObjectId('68277c3f541264f156e6f1da'), ObjectId('68277c3f541264f156e6f1db'), ObjectId('68277c3f541264f156e6f1dc'), ObjectId('68277c3f541264f156e6f1dd'), ObjectId('68277c3f541264f156e6f1de'), ObjectId('68277c3f541264f156e6f1df'), ObjectId('68277c3f541264f156e6f1e0'), ObjectId('68277c3f541264f156e6f1e1'), ObjectId('68277c3f541264f156e6f1e2'), ObjectId('68277c3f541264f156e6f1e3'), ObjectId('68277c3f541264f156e6f1e4'), ObjectId('68277c3f541264f156e6f1e5'), ObjectId('68277c3f541264f156e6f1e6'), ObjectId('68277c3f541264f156e6f1e7'), ObjectId('68277c3f541264f156e6f1e8'), ObjectId('68277c3f541264f156e6f1e9'), ObjectId('68277c3f541264f156e6f1ea'), ObjectId('68277c3f541264f156e6f1eb'), ObjectId('68277c3f541264f156e6f1ec'), ObjectId('68277c3f541264f156e6f1ed'), ObjectId('68277c3f541264f156e6f1ee'), ObjectId('68277c3f541264f156e6f1ef'), ObjectId('68277c3f541264f156e6f1f0'), ObjectId('68277c3f541264f156e6f1f1'), ObjectId('68277c3f541264f156e6f1f2'), ObjectId('68277c3f541264f156e6f1

In [ ]:
from project.storage.pipelines import thirthy_sec_pipeline, five_min_pipeline
main_log = logs[4001]
list(col.aggregate(five_min_pipeline(main_log)))

[{'_id': None,
  'log_count': 1654,
  'success_rate': 0.9540507859733979,
  'bash_ratio': 0.6457073760580411}]

In [3]:
from src.data.features.event_feature_extractor import EventFeatureExtractor

fe = EventFeatureExtractor()

/home/daiv8314/env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cpu


In [15]:
def get_labels():
    agg_feature_labels = [
        'log_count',
        'cwd_avg_risk_score',
        'avg_arg_count',
        'avg_flag_count',
        'bash_count_rate',
        'success_rate',
        'unique_pids',
    ]
    agg_command_labels = [f'avg_embedded_command_{i}' for i in range(10)]


    five_min_feature_labels = [f'five_min_{label}' for label in agg_feature_labels]
    thirty_sec_feature_labels = [f'thirty_sec_{label}' for label in agg_feature_labels]

    five_min_command_labels = [f'five_min_{label}' for label in agg_command_labels]
    thirty_sec_command_labels = [f'thirty_sec_{label}' for label in agg_command_labels]
    event_command_labels = [f'cur_event_{label}' for label in agg_command_labels]

    event_feature_labels = [
        'success',
    ]

    return event_command_labels + event_feature_labels + thirty_sec_command_labels + thirty_sec_feature_labels + five_min_command_labels + five_min_feature_labels

def get_selected_labels():
    labels = get_labels()
    idxs = [25, 15, 11, 18, 19, 20, 21, 16, 14, 13, 12, 26, 17, 0, 6, 27, 32, 42, 33, 9, 7, 4, 2, 43, 5, 38, 3, 1, 8]
    selected_labels = [labels[i] for i in idxs]
    return selected_labels

In [16]:
get_selected_labels()

['thirty_sec_bash_count_rate',
 'thirty_sec_avg_embedded_command_4',
 'thirty_sec_avg_embedded_command_0',
 'thirty_sec_avg_embedded_command_7',
 'thirty_sec_avg_embedded_command_8',
 'thirty_sec_avg_embedded_command_9',
 'thirty_sec_log_count',
 'thirty_sec_avg_embedded_command_5',
 'thirty_sec_avg_embedded_command_3',
 'thirty_sec_avg_embedded_command_2',
 'thirty_sec_avg_embedded_command_1',
 'thirty_sec_success_rate',
 'thirty_sec_avg_embedded_command_6',
 'cur_event_avg_embedded_command_0',
 'cur_event_avg_embedded_command_6',
 'thirty_sec_unique_pids',
 'five_min_avg_embedded_command_4',
 'five_min_bash_count_rate',
 'five_min_avg_embedded_command_5',
 'cur_event_avg_embedded_command_9',
 'cur_event_avg_embedded_command_7',
 'cur_event_avg_embedded_command_4',
 'cur_event_avg_embedded_command_2',
 'five_min_success_rate',
 'cur_event_avg_embedded_command_5',
 'five_min_log_count',
 'cur_event_avg_embedded_command_3',
 'cur_event_avg_embedded_command_1',
 'cur_event_avg_embedded_c

In [9]:
import requests

url = 'http://localhost:8000/predict'
data = logs[4001]

response = requests.post(url, json=data)
response.json()